In [1]:
import polars as pl

In [2]:
train_df = pl.read_csv("./fold/train_with_fold_4.csv")
print(train_df.shape)

(401059, 56)


In [11]:
((train_df.select("tbp_lv_H") + 2) / 110).min()

tbp_lv_H
f64
0.003871


In [18]:
# targetが1の患者のpatient_idを取得
target_one_patient_ids = train_df.filter(pl.col("target") == 1)["patient_id"].unique()

# 対象のpatient_idを持つ行のみをフィルタリング
train_df = train_df.filter(pl.col("patient_id").is_in(target_one_patient_ids))

In [19]:
train_df.shape

(156004, 57)

In [20]:
train_df = train_df.with_columns(
    (pl.col("lesion_id").is_not_null().cast(pl.Int8)).alias("has_lesion_id")
)

In [21]:
train_df["has_lesion_id"].describe()

statistic,value
str,f64
"""count""",156004.0
"""null_count""",0.0
"""mean""",0.03489
"""std""",0.183502
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",1.0


In [7]:
train_df["target"].describe()

statistic,value
str,f64
"""count""",156004.0
"""null_count""",0.0
"""mean""",0.002519
"""std""",0.050128
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",1.0


In [9]:
import pandas as pd

In [10]:
train_df2 = pd.read_csv("./fold/train_with_fold_4.csv")
print(train_df2.shape)

/tmp/ipykernel_164592/764784266.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df2 = pd.read_csv("./fold/train_with_fold_4.csv")


(401059, 56)


In [22]:
train_df2["has_lesion_id"] = train_df2["lesion_id"].notnull().astype(int)

In [23]:
train_df2["has_lesion_id"]

0         0
1         1
2         0
3         0
4         0
         ..
401054    1
401055    0
401056    1
401057    0
401058    0
Name: has_lesion_id, Length: 401059, dtype: int64

In [1]:
import timm

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import timm

# 利用可能なモデルのリストを取得
model_names = timm.list_models()

# ConvNeXt V2 系モデルをフィルタリング
convnextv2_models = [name for name in model_names if 'convnext' in name]

# 利用可能な ConvNeXt V2 系モデルを表示
print(convnextv2_models)

['convnext_atto', 'convnext_atto_ols', 'convnext_base', 'convnext_femto', 'convnext_femto_ols', 'convnext_large', 'convnext_large_mlp', 'convnext_nano', 'convnext_nano_ols', 'convnext_pico', 'convnext_pico_ols', 'convnext_small', 'convnext_tiny', 'convnext_tiny_hnf', 'convnext_xlarge', 'convnext_xxlarge', 'convnextv2_atto', 'convnextv2_base', 'convnextv2_femto', 'convnextv2_huge', 'convnextv2_large', 'convnextv2_nano', 'convnextv2_pico', 'convnextv2_small', 'convnextv2_tiny']


In [14]:
import timm

# ConvNeXt Small モデルの事前学習タグを取得
model_name = 'convnextv2_base'
pretrained_tags = timm.list_pretrained(model_name)

print(f"Pretrained tags for {model_name}:")
for tag in pretrained_tags:
    print(tag)

Pretrained tags for convnextv2_base:
convnextv2_base.fcmae
convnextv2_base.fcmae_ft_in1k
convnextv2_base.fcmae_ft_in22k_in1k
convnextv2_base.fcmae_ft_in22k_in1k_384


In [9]:
timm.create_model('convnextv2_small')

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): GlobalResponseNormMlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (grn): GlobalResponseNorm()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), 

In [1]:
import pandas as pd

In [5]:
df_past = pd.read_csv("../data/metadata.csv", low_memory=False)
df_new = pd.read_csv("../data/train-metadata.csv", low_memory=False)

In [6]:
df_past.shape

(81722, 28)

In [7]:
df_new.shape

(401059, 55)

In [8]:
"""

        self.df["age_scaled"] = (
            self.df["age_approx"].fillna(60) / 90
        )  # 後でloglossで評価するのでクソでかい数字で割っておく
    self.df["sex"] = self.df["sex"].fillna("male")
    self.df["anatom_site_general"] = self.df["anatom_site_general"].fillna("unk")
    self.df["has_lesion_id"] = self.df["lesion_id"].notnull().astype(int)
    # 最小値が-1.5くらい、最大値が105くらいなので[0, 1]に収まるように
    self.df["tbp_lv_H"] = (self.df["tbp_lv_H"] + 2) / 110

    # 0fill
    self.df = self.df.infer_objects()  # これやっとかないと警告が出る
    self.df = self.df.fillna(0)
    self.df["anatom_site_general_enc"] = self.df["anatom_site_general"].map(
        ANATOM_SITE_GENERAL_ENCODER
    )
    self.df["sex_enc"] = self.df["sex"].map(SEX_ENCODER)
    """

'\n\n        self.df["age_scaled"] = (\n            self.df["age_approx"].fillna(60) / 90\n        )  # 後でloglossで評価するのでクソでかい数字で割っておく\n    self.df["sex"] = self.df["sex"].fillna("male")\n    self.df["anatom_site_general"] = self.df["anatom_site_general"].fillna("unk")\n    self.df["has_lesion_id"] = self.df["lesion_id"].notnull().astype(int)\n    # 最小値が-1.5くらい、最大値が105くらいなので[0, 1]に収まるように\n    self.df["tbp_lv_H"] = (self.df["tbp_lv_H"] + 2) / 110\n\n    # 0fill\n    self.df = self.df.infer_objects()  # これやっとかないと警告が出る\n    self.df = self.df.fillna(0)\n    self.df["anatom_site_general_enc"] = self.df["anatom_site_general"].map(\n        ANATOM_SITE_GENERAL_ENCODER\n    )\n    self.df["sex_enc"] = self.df["sex"].map(SEX_ENCODER)\n    '

In [10]:
print(df_past["age_approx"].describe())
print(df_new["age_approx"].describe())

count    79702.000000
mean        47.849489
std         19.865311
min          0.000000
25%         35.000000
50%         50.000000
75%         65.000000
max         85.000000
Name: age_approx, dtype: float64
count    398261.000000
mean         58.012986
std          13.596165
min           5.000000
25%          50.000000
50%          60.000000
75%          70.000000
max          85.000000
Name: age_approx, dtype: float64


In [19]:
print(df_past["sex"].value_counts(normalize=True))
print(df_new["sex"].value_counts(normalize=True))

sex
male      0.538947
female    0.461053
Name: proportion, dtype: float64
sex
male      0.681688
female    0.318312
Name: proportion, dtype: float64


In [18]:
print(df_past["anatom_site_general"].value_counts(normalize=True))
print(df_new["anatom_site_general"].value_counts(normalize=True))

anatom_site_general
lower extremity    0.249803
anterior torso     0.216224
posterior torso    0.210664
upper extremity    0.155275
head/neck          0.135922
lateral torso      0.016002
palms/soles        0.013052
oral/genital       0.003058
Name: proportion, dtype: float64
anatom_site_general
posterior torso    0.308376
lower extremity    0.260630
anterior torso     0.222032
upper extremity    0.178488
head/neck          0.030473
Name: proportion, dtype: float64


In [22]:
print(df_past["lesion_id"].isnull().mean())
print(df_new["lesion_id"].isnull().mean())

0.2172731944886322
0.945000610882688


In [25]:
#print(df_past["tbp_lv_H"].isnull().mean())
print(df_new["tbp_lv_H"].mean())

54.65368914353473


In [27]:
pd.concat([df_new, df_past], axis=0)

,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,family_hx_mm,fitzpatrick_skin_type,mel_class,mel_type,mel_ulcer,melanocytic,nevus_type,personal_hx_mm,pixels_x,pixels_y
0,ISIC_0015670,0.0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ISIC_0015845,0.0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ISIC_0015864,0.0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ISIC_0015902,0.0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ISIC_0024200,0.0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81717,ISIC_0000004,NaN,NaN,80.0,male,posterior torso,NaN,dermoscopic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1022.0,767.0
81718,ISIC_0000003,NaN,NaN,30.0,male,upper extremity,NaN,dermoscopic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1022.0,767.0
81719,ISIC_0000002,NaN,NaN,60.0,female,upper extremity,NaN,dermoscopic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1022.0,767.0
81720,ISIC_0000001,NaN,NaN,30.0,female,anterior torso,NaN,dermoscopic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,1022.0,767.0
